In [0]:
%sh
#To keep it simple, we'll download and extract the dataset using standard bash commands 
#Install 7zip to extract the file
apt-get install -y p7zip-full

rm -rf /tmp/quant || true
mkdir -p /tmp/quant
cd /tmp/quant
#Download & extract the quant archive
curl -L https://archive.org/download/stackexchange/quant.stackexchange.com.7z -o quant.7z
7z x quant.7z 
#Move the dataset to our main bucket
rm -rf /dbfs/dbdemos/product/llm/quant/raw || true
mkdir -p /dbfs/dbdemos/product/llm/quant/raw
cp -f Posts.xml /dbfs/dbdemos/product/llm/quant/raw

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 1549 kB of archives.
After this operation, 5847 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 p7zip amd64 16.02+dfsg-8 [363 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 p7zip-full amd64 16.02+dfsg-8 [1186 kB]


debconf: delaying package configuration, since apt-utils is not installed


Fetched 1549 kB in 3s (473 kB/s)
Selecting previously unselected package p7zip.
(Reading database ... 100130 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-8_amd64.deb ...
Unpacking p7zip (16.02+dfsg-8) ...
Selecting previously unselected package p7zip-full.
Preparing to unpack .../p7zip-full_16.02+dfsg-8_amd64.deb ...
Unpacking p7zip-full (16.02+dfsg-8) ...
Setting up p7zip (16.02+dfsg-8) ...
Setting up p7zip-full (16.02+dfsg-8) ...
Processing triggers for man-db (2.10.2-1) ...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 46.1M  100 46.1M    0     0  4034k      0  0:00:11  0:00:11 --:--:-- 5008k



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,32 CPUs AMD EPYC 7R32 (830F10),ASM,AES-NI)

Scanning the drive for archives:
1 file, 48416494 bytes (47 MiB)

Extracting archive: quant.7z
--
Path = quant.7z
Type = 7z
Physical Size = 48416494
Headers Size = 333
Method = BZip2
Solid = -
Blocks = 8

Everything is Ok

Files: 8
Size:       266808589
Compressed: 48416494


In [0]:
%fs ls /dbdemos/product/llm/quant/raw

path,name,size,modificationTime
dbfs:/dbdemos/product/llm/quant/raw/Posts.xml,Posts.xml,73662724,1687046644000


In [0]:
quant_raw_path = "/dbdemos/product/llm/quant/raw"
print(f"loading raw xml dataset under {quant_raw_path}")
raw_quant = spark.read.format("xml").option("rowTag", "row").load(f"{quant_raw_path}/Posts.xml")
raw_quant.show(10)

loading raw xml dataset under /dbdemos/product/llm/quant/raw
+-----------------+------------+--------------------+--------------------+-------------+--------------------+---------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|         _ClosedDate|_CommentCount| _CommunityOwnedDate|_ContentLicense|       _CreationDate|_FavoriteCount|_Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+--------------------+-------------+--------------------+---------------+--------------------+--------------+---+--------------------+--------------------+--------

In [0]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import col, udf, length, pandas_udf

#UDF to transform html content as text
@pandas_udf("string")
def html_to_text(html):
  return html.apply(lambda x: BeautifulSoup(x).get_text())

quant_df =(raw_quant
                  .filter("_Score >= 5") # keep only good answer/question
                  .filter(length("_Body") <= 1000) #remove too long questions
                  .withColumn("text", html_to_text("_Body")) #Convert html to text
                  .withColumnsRenamed({"_Id": "id", "_ParentId": "parent_id"})
                  .select("id", "text", "parent_id"))

quant_df.show(10)

+---+--------------------+---------+
| id|                text|parent_id|
+---+--------------------+---------+
|  1|To get the ball r...|     null|
|  2|I like Statistics...|        1|
|  3|I want to start l...|     null|
|  4|John C. Hull's "O...|        1|
|  5|How do you model ...|     null|
|  6|This may be too b...|        1|
|  8|I like the follow...|        1|
|  9|It seems that VIX...|     null|
| 10|Clark,\nThis is o...|        1|
| 11|Options, Futures,...|        1|
+---+--------------------+---------+
only showing top 10 rows



In [0]:
docs_df = quant_df.withColumn('text_length', length(col('text')))\
                    .orderBy(col('text_length').desc()).limit(10)\
                    .select('text','text_length')
docs_df.show(10)

+--------------------+-----------+
|                text|text_length|
+--------------------+-----------+
|Co-integration is...|        986|
|Well, what you fi...|        983|
|VIX is calculated...|        980|
|The answer to you...|        977|
|The estimation of...|        975|
|Recently I've rea...|        970|
|According to my u...|        967|
|Buy copies of Bre...|        966|
|There have been n...|        965|
|Assuming you avoi...|        963|
+--------------------+-----------+



In [0]:
from typing import Iterator
import pandas as pd 
from transformers import pipeline
import torch

#Make sure we clean the memory
try:
    torch.cuda.empty_cache()
    from numba import cuda
    cuda.get_current_device().reset()
except Exception as e:
    print(f"Couldn't clean the memory: {e}")

@pandas_udf("string")
def summarize(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Load the model for summarization
    torch.cuda.empty_cache()
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device="cuda:0")
    def summarize_txt(text):
      return summarizer(text)[0]['summary_text']

    for serie in iterator:
        # get a summary for each row
        yield serie.apply(summarize_txt)

docs_df = docs_df.repartition(1)\
                 .withColumn("summary", summarize("text"))\
                 .withColumn('summary_length', length(col('summary')))
display(docs_df)

text,text_length,summary,summary_length
"Co-integration is a measure / indicator of the long running relationship between 2 or more time series. A short answer to how you can use it, is the pairs trading strategy or in Econometrics can be used to formulate a regression. using the classic example, you can use 2 stocks like Coke (C) and Pepsi (P) (or commodities such as Gold and Silver) in a pairs trading strategy. Your strategy will involve first finding out if the stocks are co-integrated; if they are then you will need to have a strat like: - If the spread (C - P) > threshold then sell C and buy P - If the spread (C - P) < threshold then buy C and sell P The idea here is that if the spread widens say C increases then eventually P will also increase or C will eventually revert to some long running value. The key challenge here is to determine when the spread is at its optimal value, so that you know when to enter / exit the trade Very quick and basic, there are tons of info on this strat on the web.",986,Co-integration is a measure / indicator of the long running relationship between 2 or more time series . The idea here is that if the spread widens say C increases then eventually P will also increase or C will eventually revert to some long running value . The key challenge is to determine when the spread is at its optimal value .,334
"Well, what you find is that the introduction of stochastic vol changes the delta of your options. So what does this mean? If the new delta reduces the variance of your hedged portfolio versus the pure local vol model , then it means that the introduction of stochastic vol has resulted in a better description of market dynamics versus the pure local vol model. Secondly, what you also find is that you can have different models all of which reprice the vanilla options, but that some exotic options have very different prices in the different models. For example , the introduction of stochastic vol can be done in a way that preserves the vanilla option prices , but it lowers the value of forward implied volatilities in the model versus a simple local vol model. Thus, exotics that depend on forward vols ( cliques, Bermudan etc) are priced very differently. Hence another reason to introduce stochastic vol is to improve the pricing of exotics, given the vanilla market.",983,"Stochastic vol has resulted in a better description of market dynamics versus the pure local vol model . Exotics that depend on forward vols ( cliques, Bermudan etc) are priced very differently . Another reason to introduce stochastic vol is to improve the pricing of exotics, given the vanilla market .",304
"VIX is calculated from a basket of SPX options, and VIX futures expire into following expiration, e.g. September VIX futures that will expire next Wednesday will use SPX October options chain to calculate settlement value. If $B$ is the value of the basket then VIX value at expiration is $\sqrt{ B }$. Then VIX futures price is the expectation of the basket $VIX _{F} = E[\sqrt{ B }]$. Delta of the VIX futures price with respect to the basket would be $$\ \frac{\partial VIX _{F}}{\partial B} = \frac{\partial E[\sqrt{ B }]}{\partial B}$$ As you can see that taking that expectation is not simple, since there is no simple connection between VIX futures greeks and SPX options greeks because of the expectation and square root. So ""use the chain rule and linearity of the derivative"" approach would not get you anywhere. But that does not mean that such derivative is 0. Such derivative can be calculated in Malliavin sense, but that is probably not what you're looking for.",980,"VIX is calculated from a basket of SPX options, and VIX futures expire into following expiration . If $B$ is the value of the basket then VIX value at expiration is $\sqrt{ B }$. If VIX price is the expectation of basket $VIX _{F} = E[\sqrt { B }]$. Delta of the Vix futures price with respect to the basket would be$$\ \frac{\partial VIX _ {F]{\partial B} = \frac